In [ ]:

import pandas as pd
import numpy as np
import requests
from IPython.display import HTML
from transformers import  AutoModelForMaskedLM, AutoTokenizer
import seaborn as sn
import matplotlib.pyplot as plt
from datasets import Dataset, load_dataset

In [2]:
pip install -U datasets

In [ ]:
from datasets import DatasetDict
ds = load_dataset("Czunzun/Reuters_first_512")

train_test = ds.train_test_split(test_size=0.2)  # 80% train, 20% test
ds = DatasetDict({
    'train': train_test['train'],
    'test': train_test['test']
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/413 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/53.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/97466 [00:00<?, ? examples/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['first_512', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 31188
    })
    test: Dataset({
        features: ['first_512', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7798
    })
})

In [8]:
from huggingface_hub import login
login()

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

#Importing the models
bert_model = AutoModelForMaskedLM.from_pretrained("distilbert/distilbert-base-uncased")

#Importing BERT tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

#Importing Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer,
    mlm=True,
    mlm_probability=.15
)





#defining training arguments
default_args = {
    "evaluation_strategy": "steps",
    "num_train_eophs": 1,
    "log_level": "error",
    "report_to": "none",
    
}



#Calling Trainingarg


#Optimizing for speed, the more data we can feed the model the better

#Only running at about 40% of data wihch was about 27000 artciles and took 30 minutes to train on a T5e-1 TPU

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    fp16=True,
    optim="",
    push_to_hub=True,
    hub_model_id="finbert2_v2",
    **default_args
    )


#defining trainer
trainer = Trainer(
    model=bert_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    tokenizer=bert_tokenizer
)
trainer.train()


Some weights of DistilBertForMaskedLM were not initialized from the model checkpoint at distilbert/distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-9-441211430.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


HfHubHTTPError: (Request ID: Root=1-6859b9f1-5fea78db258000b600ef675c;5aa52686-43e6-4499-8961-996d9704887f)

403 Forbidden: You don't have the rights to create a model under the namespace "Czunzun".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.